In [1]:
%%writefile sieve.starter.c
/* Blue Waters Petascale Semester Curriculum v1.0
 * Unit 4: OpenMP
 * Lesson 9: Lesson 9: Sieve of Eratosthenes
 * File: sieve.starter.c
 * Developed by Aaron Weeden for the Shodor Education Foundation, Inc.
 * Adapted by David A. Joiner for the Shodor Education Foundation, Inc.
 *
 * Copyright (c) 2020 The Shodor Education Foundation, Inc.
 *
 * Browse and search the full curriculum at
 * <http://shodor.org/petascale/materials/semester-curriculum>.
 *
 * We welcome your improvements! You can submit your proposed changes to this
 * material and the rest of the curriculum in our GitHub repository at
 * <https://github.com/shodor-education/petascale-semester-curriculum>.
 *
 * We want to hear from you! Please let us know your experiences using this
 * material by sending email to petascale@shodor.org
 *
 * This program is free software: you can redistribute it and/or modify
 * it under the terms of the GNU Affero General Public License as published
 * by the Free Software Foundation, either version 3 of the License, or
 * (at your option) any later version.
 *
 * This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU Affero General Public License for more details.
 *
 * You should have received a copy of the GNU Affero General Public License
 * along with this program.  If not, see <https://www.gnu.org/licenses/>.
 */

/* Starter code
 *  -- to run, use ./sieve.starter -n N, where N is the value under which to find
 *     primes.
 */
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>

#define MIN(a,b) (((a)<(b))?(a):(b))
#define MAX(a,b) (((a)>(b))?(a):(b))

int main(int argc, char **argv) {
    /* Declare variables */
    long N = 16; /* The positive integer under which we are finding primes */
    long sqrtN = 4; /* The square root of N, which is stored in a variable to 
                      avoid making excessive calls to sqrt(N) */
    long c = 2; /* Used to check the next number to be circled */
    long m = 3; /* Used to check the next number to be marked */
    long *list; /* The list of numbers -- if list[x] equals 1, then x is marked. 
                  If list[x] equals 0, then x is unmarked. */
    char next_option = ' '; /* Used for parsing command line arguments */
    double start, end;
    long counter;
   
    /* Parse command line arguments -- enter 'man 3 getopt' on a shell to see
       how this works */
    while((next_option = getopt(argc, argv, "n:")) != -1) {
        switch(next_option) {
            case 'n':
                N = atoi(optarg);
                break;
            case '?':
            default:
                fprintf(stderr, "Usage: %s [-n N]\n", argv[0]);
                exit(-1);
        }
    }

    printf("Solving for n = %ld\n",N);


    /* Calculate sqrtN */
    sqrtN = (long)sqrt(N);

    /* Allocate memory for list */
    list = (long*)malloc(N * sizeof(long));

    /* Exit if malloc failed */
    if(list == NULL) {
        fprintf(stderr, "Error: Failed to allocate memory for list.\n");
        exit(-1);
    }

    start = omp_get_wtime();
    /* Run through each number in the list */
    for(c = 2; c <= N-1; c++) {

        /* Set each number as unmarked */
        list[c] = 0;
    }

    /* Run through each number in the list up through sqrtN */
    for(c = 2; c <= sqrtN; c++) {

        /* If the number is unmarked */
        if(list[c] == 0) {

            /* Run through each number bigger than c */
            for(m = c*c; m <= N-1; m+=c) {
                list[m] = 1;
            }
        }
    }
    end = omp_get_wtime();


    /* Run through each number in the list */
    counter=0;
    for(c = 2; c <= (N-1)/2 && counter < 10; c++) {

        /* If the number is unmarked */
        if(list[c] == 0) {

            /* The number is prime, print it */
            printf("%ld ", c);
            counter++;
        }
    }
    printf("\n");
        /* Run through each number in the list */
    counter=0;
    for(c = N-1; c > (N-1)/2 && counter<10; c--) {

        /* If the number is unmarked */
        if(list[c] == 0) {

            /* The number is prime, print it */
            printf("%ld ", c);
            counter++;
        }
    }
    printf("\n");
    printf("Elapsed time in calculation: %lf (s)\n",(end-start));

    /* Deallocate memory for list */
    free(list);

    return 0;
}


Writing sieve.starter.c


In [2]:

!gcc -o sieve.starter sieve.starter.c -lm -fopenmp

In [3]:
!./sieve.starter -n 100000000

Solving for n = 100000000
2 3 5 7 11 13 17 19 23 29 
99999989 99999971 99999959 99999941 99999931 99999847 99999839 99999827 99999821 99999787 
Elapsed time in calculation: 4.376531 (s)


In [1]:
%%writefile sieve.inner.c
/* Blue Waters Petascale Semester Curriculum v1.0
 * Unit 4: OpenMP
 * Lesson 9: Lesson 9: Sieve of Eratosthenes
 * File: sieve.inner.c
 * Developed by Aaron Weeden for the Shodor Education Foundation, Inc.
 * Adapted by David A. Joiner for the Shodor Education Foundation, Inc.
 *
 * Copyright (c) 2020 The Shodor Education Foundation, Inc.
 *
 * Browse and search the full curriculum at
 * <http://shodor.org/petascale/materials/semester-curriculum>.
 *
 * We welcome your improvements! You can submit your proposed changes to this
 * material and the rest of the curriculum in our GitHub repository at
 * <https://github.com/shodor-education/petascale-semester-curriculum>.
 *
 * We want to hear from you! Please let us know your experiences using this
 * material by sending email to petascale@shodor.org
 *
 * This program is free software: you can redistribute it and/or modify
 * it under the terms of the GNU Affero General Public License as published
 * by the Free Software Foundation, either version 3 of the License, or
 * (at your option) any later version.
 *
 * This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU Affero General Public License for more details.
 *
 * You should have received a copy of the GNU Affero General Public License
 * along with this program.  If not, see <https://www.gnu.org/licenses/>.
 */

/* Solution code
 *  -- to run, use ./sieve.inner -n N, where N is the value under which to find
 *     primes.
 */
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>

#define MIN(a,b) (((a)<(b))?(a):(b))
#define MAX(a,b) (((a)>(b))?(a):(b))

int main(int argc, char **argv) {
    /* Declare variables */
    long N = 16; /* The positive integer under which we are finding primes */
    long sqrtN = 4; /* The square root of N, which is stored in a variable to 
                      avoid making excessive calls to sqrt(N) */
    long c = 2; /* Used to check the next number to be circled */
    long m = 3; /* Used to check the next number to be marked */
    long *list; /* The list of numbers -- if list[x] equals 1, then x is marked. 
                  If list[x] equals 0, then x is unmarked. */
    char next_option = ' '; /* Used for parsing command line arguments */
    double start, end;
    long counter;
   
    /* Parse command line arguments -- enter 'man 3 getopt' on a shell to see
       how this works */
    while((next_option = getopt(argc, argv, "n:")) != -1) {
        switch(next_option) {
            case 'n':
                N = atoi(optarg);
                break;
            case '?':
            default:
                fprintf(stderr, "Usage: %s [-n N]\n", argv[0]);
                exit(-1);
        }
    }

    printf("Solving for n = %ld\n",N);


    /* Calculate sqrtN */
    sqrtN = (long)sqrt(N);

    /* Allocate memory for list */
    list = (long*)malloc(N * sizeof(long));

    /* Exit if malloc failed */
    if(list == NULL) {
        fprintf(stderr, "Error: Failed to allocate memory for list.\n");
        exit(-1);
    }

    start = omp_get_wtime();
    /* Run through each number in the list */
    for(c = 2; c <= N-1; c++) {

        /* Set each number as unmarked */
        list[c] = 0;
    }

    /* Run through each number in the list up through sqrtN */
    for(c = 2; c <= sqrtN; c++) {

        /* If the number is unmarked */
        if(list[c] == 0) {

            /* Run through each number bigger than c */
#pragma omp parallel for schedule(static)
            for(m = c*c; m <= N-1; m+=c) {
                list[m] = 1;
            }
        }
    }
    end = omp_get_wtime();


    /* Run through each number in the list */
    counter=0;
    for(c = 2; c <= (N-1)/2 && counter < 10; c++) {

        /* If the number is unmarked */
        if(list[c] == 0) {

            /* The number is prime, print it */
            printf("%ld ", c);
            counter++;
        }
    }
    printf("\n");
        /* Run through each number in the list */
    counter=0;
    for(c = N-1; c > (N-1)/2 && counter<10; c--) {

        /* If the number is unmarked */
        if(list[c] == 0) {

            /* The number is prime, print it */
            printf("%ld ", c);
            counter++;
        }
    }
    printf("\n");
    printf("Elapsed time in calculation: %lf (s)\n",(end-start));

    /* Deallocate memory for list */
    free(list);

    return 0;
}

Writing sieve.inner.c


In [5]:
!gcc -o sieve.inner sieve.inner.c -lm -fopenmp

In [6]:
!./sieve.starter -n 200000000
!./sieve.inner  -n 200000000

Solving for n = 200000000
tcmalloc: large alloc 1600004096 bytes == 0x5558f335c000 @  0x7f2433e1c1e7 0x5558f1c8bac8 0x7f243344cb97 0x5558f1c8b90a
2 3 5 7 11 13 17 19 23 29 
199999991 199999963 199999957 199999949 199999931 199999903 199999901 199999889 199999853 199999841 
Elapsed time in calculation: 8.808446 (s)
Solving for n = 200000000
tcmalloc: large alloc 1600004096 bytes == 0x55a1eda1e000 @  0x7f3d054021e7 0x55a1eb25cc77 0x7f3d04a32b97 0x55a1eb25caaa
2 3 5 7 11 13 17 19 23 29 
199999991 199999963 199999957 199999949 199999931 199999903 199999901 199999889 199999853 199999841 
Elapsed time in calculation: 5.638280 (s)


#Student Learning Objectives

As a result of this exercise students will

* demonstrate ability to analyze a problem with nested loops and discuss instancces of loop carried dependencies.
* make decisions about where and how to parallelize a nested loop in OpenMP
* implement a #pragma omp parallel for statement with appropriate scheduling.
* profile a code using OpenMP timers
* show proficiency at using the classic Sieve of Eratosthenes algorithm to compute prime numbers

#Student Instructions

The Sieve of Eratosthenes is a classic $n \log(n)$ algorithm for determining a large list of prime numbers, and if your goal is to calculat a complete and comprehensive list of primes up to some value, is likely the fastest algorithm that you can use.

It works as follows. You start with a list of all numbers up to N. (For our computer algorithm we will in practice count to N-1 to make our array syntax easier). For each of these numbers, construct a list of whether the number is or is not prime. In the beginning, assume all numbers are prime until proven otherwise. This might be most easily done with an array of ints or booleans, so that <PRE>list[5] == 0</PRE> would imply that 5 is prime, whereas <PRE>list[4] == 1</PRE> would imply that 4 is not.

You have been provided a starter code that implements the Sieve of Eratosthenes algorithm in C, with additional includes and profiling instructions for OpenMP. Your goal is to discuss, plan, and implement a parallelization of the problem in OpenMP, and test the performance of your implementation.

1. Consider the starter code. Compile and run it as you would for any OpenMP program. <PRE>gcc -o sieve.starter sieve.starter.c -lm -fopenmp</PRE>  For increasing list lengths, note the running time. Does the algorithm scale as expected?

2. For the starter code, note the amount of time required to build a list. Before attempting parallelization, make a hypothesis as to how long the list might need to be before you begin to see the benefit of parallelism.

3. Looking at the nested loop, determine whether there are any loop carried dependencies in either the inner or outer loop. What is the implication of a loop carried dependency in parallelization?

4. Implement a plan to parallelize using OpenMP. Focus on the outermost loop without a loop carried dependency. Parallelize your code.

5. Using your parallel code, compare parallel efficiency as you increase the problem size and number of threads. Display your results in a table, and discuss the effectiveness of parallelization for this problem at different problem sizes.


#Teacher Instructions

Pose the problem to the students, and descibe how the Sieve algorithm works. There are many different examples out there, including:

https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes

http://www.shodor.org/petascale/materials/UPModules/sieveOfEratosthenes/

Introduce the students to the starter code. Note that the Sieve algorithm is a classic one for predicting lists of contiguous primes, and is already very fast. As they discuss running times and hypothesize how parallelization will improve the code, have them keep in mind realistic expectations. Codes that run in milliseconds, including all overhead and I/O, are not likely to show speedup in parallel.

Note the use of the OpenMP specific timers. Remind the students that standard clock timers that they might use to profile serial codes often work unexpectedly in parallel codes--leading to each of the different parallel programming libraries to typically implement their own timers. If students have already perfromed timing using other methods, have them compare and contrast OpenMP's omp_get_wtime method with other ways they have learned.

As they discuss the loop carried dependencies, they should see that each pass through the outer loop eliminates more sections within the inner loop. While technically this can be done concurrently without producing wrong output, much of the speedup of the algorithm comes from being able to skip numbers that have already been determined not prime--thus parallelizing the outer loop does carry a loop carried dependency. The inner loop does not, however students may see efficiency loss due to many threads accessing nearby elements of memory in the variable "list." If students have already discussed false sharing, you might note for them the possibility of false sharing in the parallelization of the inner loop.

The exact values of N for which the students will see improvement in parallel is machine dependent, but you should expect that value to be large, approaching the value of MAX_INT on most machines. The starter code has thus been programmed to use longs instead of ints. As N is made larger, depending on compiler and machine you may see warnings that you are allocating large arrays, or you may fail to allocate enough memory to solve the problem. This also will be highly machine and compiler dependent, so practice what the students will do ahead of time.

When the students have completed the activity, they should be able to run performance testing, and they will see that for small N the code does not benefit and is perhaps worsened by parallelism, but that as N grows the benefit of paralleism can be seen. They should not expect to see strong scaling for this example, as this is an efficient $n \log(n)$ serial algorithm already, and by the time the problem has grown to a size where parallelism is useful they will also likely be approaching machine and memory limits.
